In [1]:
import sys
# !conda install --yes pillow==6
# !conda install jupyterlab==0.32.0
import torch
from torch import nn
from torchvision import models, datasets, transforms
import time
from tqdm.auto import tqdm

In [2]:
def set_requires_grad(model, value=False):
    for param in model.parameters():
        param.requires_grad = value

In [3]:
num_classes = 100
input_size = 224
batch_size = 16

In [5]:
resnet18_model = models.resnet18(pretrained=True)

num_ftrs = resnet18_model.fc.in_features
resnet18_model.fc = nn.Linear(num_ftrs, num_classes)

# tmp_model = models.resnet18(pretrained=True)
# imagenet_fc = tmp_model.fc

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [5]:
normalize = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [6]:
trainset_100 = datasets.CIFAR100(root='./data', train=True, download=True, transform=normalize)
trainloader_100 = torch.utils.data.DataLoader(trainset_100, batch_size=batch_size, shuffle=True, num_workers=2)

testset_100 = datasets.CIFAR100(root='./data', train=False, download=True, transform=normalize)
testloader_100 = torch.utils.data.DataLoader(testset_100, batch_size=batch_size, shuffle=False, num_workers=2)

loaders = {'train': trainloader_100, 'val': testloader_100}

Files already downloaded and verified
Files already downloaded and verified


In [7]:
device = torch.device("cuda:0")

In [8]:
# googlenet_model.eval()

resnet18_model = resnet18_model.to(device)
pretrain_optimizer = torch.optim.SGD(resnet18_model.fc.parameters(), lr=0.001, momentum=0.9)
train_optimizer = torch.optim.SGD(resnet18_model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [9]:
# train

def train_model(model, dataloaders, criterion, optimizer, num_epochs=3):
    since = time.time()

    acc_history = {'train': [], 'val': []}
    loss_history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            n_batches = len(dataloaders[phase])
            for inputs, labels in tqdm(dataloaders[phase], total=n_batches):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            loss_history[phase].append(epoch_loss)
            acc_history[phase].append(epoch_acc)

            if epoch % 3 == 0:
                torch.save(model, "./data/resnet18_model{}.pth".format(epoch))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return model, acc_history

In [10]:
train_model(resnet18_model, loaders, criterion, train_optimizer, num_epochs=2)

Epoch 0/4
----------

train Loss: 1.6558 Acc: 0.0156

val Loss: 1.5336 Acc: 0.0212

Epoch 1/4
----------

train Loss: 1.6448 Acc: 0.0215

val Loss: 1.5530 Acc: 0.0257

Epoch 2/4
----------



KeyboardInterrupt: 

In [ ]:
torch.save(resnet18_model, "./data/googlenet_model.pth")



